# Partitioning the input files

ROMS requires partitioned (or tiled) input files so that the simulation can be parallelized over multiple nodes. `ROMS-Tools` can create these partitioned files for you.

In [1]:
from roms_tools.utils import partition

## Partitioning existing files

Let's assume we have already saved a global input file to disk, let's call this step 0:

0. Save global file to disk.

Here is an example for performing step 0 with `ROMS-Tools` for the grid file. (Note, however, that the `partition` tool does not care whether or whether not `ROMS-Tools` was used to perform step 0.)

In [2]:
from roms_tools import Grid

In [3]:
grid = Grid(
    nx=100, ny=100, size_x=1800, size_y=2400, center_lon=-21, center_lat=61, rot=20
)

In [4]:
path_to_global_file = "/glade/derecho/scratch/noraloose/examples/my_grid"

In [5]:
grid.save(f"{path_to_global_file}.nc")

Saving the following files:
/glade/derecho/scratch/noraloose/examples/my_grid.nc


We can now partition the saved file as follows:

1. `xr.open_dataset`: Open saved file as xarray Dataset
2. `partition`: Partition the xarray Dataset into `nx` by `ny` spatial tiles
3. `xr.save_mfdataset`: Save the partitioned xarray Datasets to file

In [6]:
import xarray as xr

In [7]:
ds = xr.open_dataset(f"{path_to_global_file}.nc")  # step 1

In [8]:
file_numbers, partitioned_datasets = partition(ds, np_eta=2, np_xi=5)  # step 2

The previous cell returns two lists:

* `file_numbers` is a list of integers representing the file numbers associated with each partitioned dataset
* `partitioned_datasets` is a list of `xarray.Dataset` objects, each representing a partitioned subdomain of the original dataset

In [9]:
file_numbers

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

For the third step, we have to create a list of paths to which to save each corresponding dataset. This is where the list `file_numbers` comes in handy.

In [11]:
paths_to_partitioned_files = [
    f"{path_to_global_file}.{file_number}.nc" for file_number in file_numbers
]

In [12]:
paths_to_partitioned_files

['/glade/derecho/scratch/noraloose/examples/my_grid.0.nc',
 '/glade/derecho/scratch/noraloose/examples/my_grid.1.nc',
 '/glade/derecho/scratch/noraloose/examples/my_grid.2.nc',
 '/glade/derecho/scratch/noraloose/examples/my_grid.3.nc',
 '/glade/derecho/scratch/noraloose/examples/my_grid.4.nc',
 '/glade/derecho/scratch/noraloose/examples/my_grid.5.nc',
 '/glade/derecho/scratch/noraloose/examples/my_grid.6.nc',
 '/glade/derecho/scratch/noraloose/examples/my_grid.7.nc',
 '/glade/derecho/scratch/noraloose/examples/my_grid.8.nc',
 '/glade/derecho/scratch/noraloose/examples/my_grid.9.nc']

In [13]:
xr.save_mfdataset(partitioned_datasets, paths_to_partitioned_files)  # step 3

The previous cell saved the ten partitioned datasets to the indicated location:

In [14]:
ls /glade/derecho/scratch/noraloose/examples/

my_grid.0.nc  my_grid.3.nc  my_grid.6.nc  my_grid.9.nc
my_grid.1.nc  my_grid.4.nc  my_grid.7.nc  my_grid.nc
my_grid.2.nc  my_grid.5.nc  my_grid.8.nc


## Creating partitioned files directly

Instead of manually going through steps 0, 1, 2, 3 above, we can also tell `ROMS-Tools` to write partitioned files directly.

In [15]:
grid = Grid(
    nx=100, ny=100, size_x=1800, size_y=2400, center_lon=-21, center_lat=61, rot=20
)

In [16]:
grid.save(filepath="/glade/derecho/scratch/noraloose/grids/my_grid", np_eta=2, np_xi=5)

Saving the following files:
/glade/derecho/scratch/noraloose/grids/my_grid.0.nc
/glade/derecho/scratch/noraloose/grids/my_grid.1.nc
/glade/derecho/scratch/noraloose/grids/my_grid.2.nc
/glade/derecho/scratch/noraloose/grids/my_grid.3.nc
/glade/derecho/scratch/noraloose/grids/my_grid.4.nc
/glade/derecho/scratch/noraloose/grids/my_grid.5.nc
/glade/derecho/scratch/noraloose/grids/my_grid.6.nc
/glade/derecho/scratch/noraloose/grids/my_grid.7.nc
/glade/derecho/scratch/noraloose/grids/my_grid.8.nc
/glade/derecho/scratch/noraloose/grids/my_grid.9.nc
